In [1]:
import os
import json
import pandas as pd
from glob import glob
from eda_support_functions import *

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_info_columns', 1000)

In [3]:
RUNS_IDS = ['20240802_224457']

In [4]:
parameters_df = load_metadata('parameters', '/Users/iliasx/Documents/GitHub/box-office-prediction/metadata')
experiment_df = load_metadata('metrics', '/Users/iliasx/Documents/GitHub/box-office-prediction/metadata')
# importance_df = load_metadata('feature_importance', '/Users/iliasx/Documents/GitHub/box-office-prediction/metadata')

In [5]:
experiment_df

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,Accuracy,Precision,Recall,F1 Score,MSE,MAPE,MAE,RMSE,RMSLE,MSLE,R2,Threshold Probability Accuracy,Threshold MAPE,Threshold MAPE (25%),ROC AUC Score
0,20240802_224457,2024-08-02 23:24:31,decision_tree_classifier,multi_class_classification,full,random_search,True,none,MinMaxScaler,0.011282,0.115336,None,0.498124,0.426014,0.498124,0.452741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20240802_224457,2024-08-02 23:36:14,lightgbm_regressor,regression,full,random_search,True,complex,RobustScaler,0.019995,20.893485,None,0.498124,0.426014,0.498124,0.452741,1.372573e+16,1.364065,6.213266e+07,1.171569e+08,1.022013,1.044510,0.656524,0.162289,1.472104,1.679541,NaN
2,20240731_163830,2024-07-31 16:55:05,random_forest_regressor,regression,full,random_search,True,complex,StandardScaler,0.002653,993.781275,None,0.498124,0.426014,0.498124,0.452741,2.453859e+16,1.673388,6.192644e+07,1.566480e+08,1.108347,1.228433,0.529389,0.156367,1.819095,2.066391,NaN
3,20240719_214914,2024-07-19 22:31:22,decision_tree_classifier,binary_classification,small_productions,random_search,True,none,RobustScaler,0.104955,0.044046,None,0.571429,0.653061,0.561404,0.603774,2.453859e+16,1.673388,6.192644e+07,1.566480e+08,1.108347,1.228433,0.529389,0.156367,1.819095,2.066391,0.573385
4,20240724_210133,2024-07-24 21:03:13,decision_tree_regressor,regression,small_productions,random_search,False,none,StandardScaler,0.155938,0.713206,None,0.571429,0.653061,0.561404,0.603774,2.595991e+15,27.918844,1.471929e+07,5.095086e+07,2.341491,5.482579,0.035311,0.098639,29.208188,31.796131,0.573385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,20240802_224457,2024-08-02 23:11:48,decision_tree_classifier,binary_classification,large_productions,random_search,True,none,StandardScaler,0.011617,0.075973,None,0.593123,0.526829,0.705882,0.603352,1.871258e+15,0.015498,3.724909e+06,4.325804e+07,0.033527,0.001124,0.956202,0.127660,2.656585,3.007394,0.605492
2536,20240729_230303,2024-07-30 00:12:45,dummy_classifier,binary_classification,medium_productions,random_search,True,none,RobustScaler,0.127511,0.401243,None,0.524116,0.476489,0.540925,0.506667,1.871258e+15,0.015498,3.724909e+06,4.325804e+07,0.033527,0.001124,0.956202,0.127660,2.656585,3.007394,0.525595
2537,20240802_224457,2024-08-02 23:15:24,logistic_regression,multi_class_classification,small_productions,random_search,False,none,RobustScaler,0.019995,0.156260,None,0.587189,0.491697,0.587189,0.533433,1.871258e+15,0.015498,3.724909e+06,4.325804e+07,0.033527,0.001124,0.956202,0.127660,2.656585,3.007394,0.525595
2538,20240724_210133,2024-07-24 21:20:18,decision_tree_classifier,multi_class_classification,medium_productions,random_search,False,none,StandardScaler,0.155938,0.783357,None,0.335726,0.475243,0.335726,0.381156,1.871258e+15,0.015498,3.724909e+06,4.325804e+07,0.033527,0.001124,0.956202,0.127660,2.656585,3.007394,0.525595


In [6]:
experiment_df.groupby('run_id').aggregate({'run_id': 'count',
                                          'dataset_name' : 'nunique',
                                          'model_type' : 'nunique',
                                          'timestamp' : 'max'}
                                          ).rename(columns={'run_id': 'count'}).sort_values('timestamp', ascending=False)

,count,dataset_name,model_type,timestamp
run_id,,,,
20240802_224457,288,4,12,2024-08-02 23:38:06
20240802_192039,2,2,1,2024-08-02 20:40:28
20240802_183731,2,2,1,2024-08-02 18:45:18
20240802_171413,2,2,1,2024-08-02 17:48:26
20240731_163830,2,2,1,2024-07-31 16:56:12
...,...,...,...,...
20240719_231427,12,2,6,2024-07-19 23:15:42
20240719_225631,2,2,1,2024-07-19 22:58:14
20240719_214914,272,4,11,2024-07-19 22:31:25


In [7]:
parameters_df = parameters_df[parameters_df['run_id'].isin(RUNS_IDS)]
experiment_df = experiment_df[experiment_df['run_id'].isin(RUNS_IDS)]

In [8]:
# parameters_df[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'model_type', 'model__units', 'model__num_layers', 'model__dropout', 'epochs','variance_threshold','class_weight']].sort_values(by=['problem_type','dataset_name','has_outliers_removed'])

In [9]:
parameters_df[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'model_type', 'scaler','variance_threshold','class_weight','C','min_child_samples','solver','num_leaves','n_estimators']].sort_values(by=['model_type','problem_type','dataset_name','has_outliers_removed'])

,problem_type,dataset_name,has_outliers_removed,feature_engineering,model_type,scaler,variance_threshold,class_weight,C,min_child_samples,solver,num_leaves,n_estimators
612,binary_classification,full,False,complex,decision_tree_classifier,StandardScaler,0.011617,balanced,0.381154,20.0,saga,30.0,100.0
884,binary_classification,full,False,none,decision_tree_classifier,MinMaxScaler,0.011282,None,0.017670,20.0,saga,226.0,1785.0
1122,binary_classification,full,True,none,decision_tree_classifier,StandardScaler,0.122331,None,12.401749,20.0,saga,237.0,1230.0
1289,binary_classification,full,True,complex,decision_tree_classifier,StandardScaler,0.011617,balanced,28.570801,20.0,saga,191.0,1230.0
425,binary_classification,large_productions,False,complex,decision_tree_classifier,StandardScaler,0.011617,balanced,0.017670,20.0,saga,252.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,regression,medium_productions,True,none,xgboost_regressor,MinMaxScaler,0.141615,balanced,12.401749,20.0,saga,90.0,971.0
1483,regression,small_productions,False,complex,xgboost_regressor,RobustScaler,0.019995,None,0.381154,20.0,saga,122.0,566.0
1794,regression,small_productions,False,none,xgboost_regressor,MinMaxScaler,0.141615,None,3.313364,20.0,saga,30.0,971.0
1263,regression,small_productions,True,none,xgboost_regressor,MinMaxScaler,0.141615,None,0.457067,20.0,saga,122.0,971.0


In [10]:
parameters_df

,run_id,timestamp,model_type,problem_type,dataset_name,grid_type,has_outliers_removed,feature_engineering,scaler,variance_threshold,duration,number_of_combinations,ccp_alpha,class_weight,criterion,max_depth,max_features,max_leaf_nodes,min_impurity_decrease,min_samples_leaf,min_samples_split,min_weight_fraction_leaf,monotonic_cst,random_state,splitter,boosting_type,colsample_bytree,importance_type,learning_rate,min_child_samples,min_child_weight,min_split_gain,n_estimators,n_jobs,num_leaves,objective,reg_alpha,reg_lambda,subsample,subsample_for_bin,subsample_freq,verbosity,bootstrap,max_samples,oob_score,verbose,warm_start,constant,strategy,quantile,C,dual,fit_intercept,intercept_scaling,l1_ratio,max_iter,multi_class,penalty,solver,tol,base_score,booster,callbacks,colsample_bylevel,colsample_bynode,device,early_stopping_rounds,enable_categorical,eval_metric,feature_types,gamma,grow_policy,interaction_constraints,max_bin,max_cat_threshold,max_cat_to_onehot,max_delta_step,max_leaves,missing,monotone_constraints,multi_strategy,num_parallel_tree,sampling_method,scale_pos_weight,tree_method,validate_parameters,copy_X,positive,build_fn,optimizer,loss,metrics,batch_size,validation_batch_size,validation_split,shuffle,run_eagerly,epochs,model__output_activation,model__loss,model__units,model__num_layers,model__dropout,num_class,break_ties,cache_size,coef0,decision_function_shape,degree,kernel,probability,shrinking,bootstrap_features,estimator__ccp_alpha,estimator__criterion,estimator__max_depth,estimator__max_features,estimator__max_leaf_nodes,estimator__min_impurity_decrease,estimator__min_samples_leaf,estimator__min_samples_split,estimator__min_weight_fraction_leaf,estimator__monotonic_cst,estimator__random_state,estimator__splitter,estimator__bootstrap,estimator__max_samples,estimator__n_estimators,estimator__n_jobs,estimator__oob_score,estimator__verbose,estimator__warm_start,model__layers_activation
0,20240802_224457,2024-08-02 23:24:31,decision_tree_classifier,multi_class_classification,full,random_search,True,none,MinMaxScaler,0.011282,0.115336,None,0.0,None,gini,9,None,None,0.0,3,23,0.0,None,42.0,best,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20240802_224457,2024-08-02 23:36:14,lightgbm_regressor,regression,full,random_search,True,complex,RobustScaler,0.019995,20.893485,None,0.0,None,gini,27,None,None,0.0,3,23,0.0,None,42.0,best,gbdt,1.0,split,0.120370,20.0,0.001,0.0,100.0,-1.0,226.0,None,0.0,0.0,1.0,200000.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,20240802_224457,2024-08-02 23:05:02,decision_tree_classifier,multi_class_classification,medium_productions,random_search,True,complex,MinMaxScaler,0.122371,1.677422,None,0.0,None,gini,26,None,None,0.0,17,28,0.0,None,42.0,best,gbdt,1.0,split,0.125660,20.0,0.001,0.0,100.0,-1.0,48.0,None,0.0,0.0,1.0,200000.0,0.0,-1.0,True,NaN,False,0,False,NaN,stratified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,20240802_224457,2024-08-02 22:49:50,xgboost_classifier,binary_classification,full,random

In [11]:
parameters_df[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering', 'n_estimators', 'max_depth', 'learning_rate', 'variance_threshold','class_weight']].sort_values(by=['problem_type','dataset_name','has_outliers_removed'])

,problem_type,dataset_name,has_outliers_removed,feature_engineering,n_estimators,max_depth,learning_rate,variance_threshold,class_weight
612,binary_classification,full,False,complex,100.0,27,0.075908,0.011617,balanced
884,binary_classification,full,False,none,1785.0,9,0.029573,0.011282,None
1278,binary_classification,full,False,none,415.0,26,0.003653,0.091850,None
1355,binary_classification,full,False,complex,1785.0,4,0.029573,0.011282,None
1412,binary_classification,full,False,complex,1219.0,3,0.032287,0.141615,None
...,...,...,...,...,...,...,...,...,...
1642,regression,small_productions,True,complex,566.0,27,0.120370,0.019995,balanced
1809,regression,small_productions,True,none,1463.0,8,0.133677,0.155938,None
1986,regression,small_productions,True,complex,1463.0,3,0.029573,0.042468,balanced
2438,regression,small_productions,True,none,100.0,30,0.075908,0.028573,None


In [12]:
# Assuming the DataFrame is named experiment_df
grouped = experiment_df[experiment_df['problem_type'] == 'regression'].groupby(
    ['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering'])

# Get the index of the minimum MAPE in each group
idx = grouped['MAPE'].idxmin()

# Select the rows that correspond to the minimum MAPE in each group
best_models = experiment_df.loc[idx]

# Display the desired columns
result_reg = best_models[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering','model_type', 'scaler', 'MAPE', 'MAE', 'R2']]

In [13]:
#pretty print
print(result_reg.to_string(index=False))

problem_type       dataset_name  has_outliers_removed feature_engineering              model_type         scaler        MAPE          MAE        R2
  regression               full                 False             complex      lightgbm_regressor StandardScaler  439.428382 6.412368e+07  0.473327
  regression               full                 False                none      lightgbm_regressor StandardScaler  560.361941 6.835668e+07  0.423402
  regression               full                  True             complex      lightgbm_regressor   RobustScaler    1.364065 6.213266e+07  0.656524
  regression               full                  True                none      lightgbm_regressor StandardScaler    1.519280 6.567530e+07  0.596499
  regression  large_productions                 False             complex       xgboost_regressor   MinMaxScaler    1.241244 1.465454e+08  0.483641
  regression  large_productions                 False                none       linear_regression StandardScaler

In [14]:
# Assuming the DataFrame is named experiment_df
grouped = experiment_df[experiment_df['problem_type'] != 'regression'].groupby(
    ['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering'])

# Get the index of the minimum MAPE in each group
idx = grouped['F1 Score'].idxmax()

# Select the rows that correspond to the minimum MAPE in each group
best_models = experiment_df.loc[idx]

# Display the desired columns
result_class = best_models[['problem_type', 'dataset_name', 'has_outliers_removed', 'feature_engineering','model_type', 'scaler', 'F1 Score']]

In [15]:
print(result_class.to_string(index=False))

              problem_type       dataset_name  has_outliers_removed feature_engineering               model_type         scaler  F1 Score
     binary_classification               full                 False             complex       xgboost_classifier   MinMaxScaler  0.664567
     binary_classification               full                 False                none random_forest_classifier StandardScaler  0.660308
     binary_classification               full                  True             complex       xgboost_classifier   MinMaxScaler  0.642857
     binary_classification               full                  True                none random_forest_classifier   RobustScaler  0.645854
     binary_classification  large_productions                 False             complex       xgboost_classifier StandardScaler  0.668367
     binary_classification  large_productions                 False                none random_forest_classifier   RobustScaler  0.666667
     binary_classification  large_

In [16]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='binary_classification', metric='F1 Score', metric_agg='max')


In [17]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='multi_class_classification', metric='F1 Score', metric_agg='max')

In [18]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='regression', metric='MAPE', metric_agg='min')

In [19]:
plot_one_metric_of_different_datasets_per_feature_engineering_outliers_with_plotly(experiment_df, problem_type='regression', metric='R2', metric_agg='max')